configuration:
models_ver - insert YOLO version's numbers that the UAP will be trained on.

epsilon, lambda_1, lambda_2 - attack's parameters. more information can be found in the [paper](https://arxiv.org/abs/2205.13618)

BDD_IMG_DIR - a path to the BDD validation set images (or any other wanted dataset)

BDD_LAB_DIR - a path to the BDD validation set labels (or any other wanted dataset)

In [14]:
models_vers = [5] # for example: models_vers = [5] or models_vers = [3, 4, 5]
epsilon = 70
lambda_1 = 1
lambda_2 = 10
seed = 42
patch_size=(640,640)
img_size=(640,640)
batch_size = 8
num_workers = 4
max_labels_per_img = 65
BDD_IMG_DIR = '/Users/coenschoof/miniconda/envs/phantomsponges/BDD100K-to-YOLOV5/bdd_in_YOLOV5_train_newLabels/images/val'
BDD_LAB_DIR = '/Users/coenschoof/miniconda/envs/phantomsponges/BDD100K-to-YOLOV5/bdd_in_YOLOV5_train_newLabels/labels/val'

Load BDD dataset:

In [15]:
import torch
import os
import random
import numpy

from datasets.augmentations1 import train_transform
from datasets.split_data_set_combined import SplitDatasetCombined_BDD

In [16]:
# batch = [(1, 'a'), (2, 'b'), (3, 'c')]

# unzipped = zip(*batch)

# numbers, letters = unzipped

# print(numbers)  # Output: (1, 2, 3)
# print(letters)  # Output: ('a', 'b', 'c')

#dus ipv [(tensor, array, string), (tensor, array, string), (tensor, array, string)] hebben we
# train_loader[0] bevat 8 images
# train_loader[1] bevat alle labels per voor 8 images
# train_loader[2] bevat paths naar 8 images
def collate_fn(batch):
    return tuple(zip(*batch))

def set_random_seed(seed_value, use_cuda=True):
    numpy.random.seed(seed_value)  # cpu vars
    torch.manual_seed(seed_value)  # cpu  vars
    random.seed(seed_value)  # Python
    os.environ['PYTHONHASHSEED'] = str(seed_value)  # Python hash buildin
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # gpu vars
        torch.backends.cudnn.deterministic = True  # needed
        torch.backends.cudnn.benchmark = False

In [17]:
split_dataset = SplitDatasetCombined_BDD(
            img_dir= BDD_IMG_DIR,
            lab_dir= BDD_LAB_DIR,
            max_lab=max_labels_per_img,
            img_size=img_size,
            transform=train_transform,
            collate_fn=collate_fn)

In [18]:
#train_loader en val_loader 
train_loader, val_loader, test_loader = split_dataset(val_split=0.1,
                                                      shuffle_dataset=True,
                                                      random_seed=seed,
                                                      batch_size=batch_size,
                                                      ordered=False,
                                                      collate_fn=collate_fn)

In [19]:
# from torch.utils.data import DataLoader, SubsetRandomSampler, RandomSampler

# sampler = RandomSampler([0,1,2,3])

# for i in sampler:
#     print(i)

In [20]:
# next(iter(train_loader))

In [21]:
# clean_xyxy_out = torch.tensor([[0.0156, 0.0312, 0.0781, 0.1094]])

# patch_xyxy_out = torch.tensor([
#                         [10/640, 20/640, 50/640, 70/640],  # [x1, y1, x2, y2]
#                         [30/640, 40/640, 80/640, 100/640],
#                         [60/640, 70/640, 120/640, 160/640],
#                     ])

# #intersect(patch_xyxy_out, clean_xyxy_out)

# torch.min(clean_xyxy_out[:, 2:].unsqueeze(1).expand(1, 3, 2), patch_xyxy_out[:, 2:].unsqueeze(0).expand(1, 3, 2))

In [22]:
# import torch
# import torchvision.transforms as transforms
# from PIL import Image

# # Load the image using PIL
# image = Image.open('/Users/coenschoof/miniconda/envs/phantomsponges/BDD100K-to-YOLOV5/bdd100k/images/100k/val/b1c9c847-3bda4659.jpg')

# # Define the transformations to apply to the image
# transform = transforms.ToTensor()

# # Apply the transformations to convert the image to a tensor
# tensor = transform(image)

In [23]:
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# # Inference
# results = model(tensor.unsqueeze(0))

In [24]:
# results.size()

In [25]:
# for (img_batch, lab_batch, _) in val_loader:
#     print(torch.stack(img_batch).size())
#     break


create UAP:

In [26]:
import numpy
from attack.uap_phantom_sponge import UAPPhantomSponge

torch.cuda.empty_cache()

patch_name = r"yolov"
for ver in models_vers:
  patch_name += f"_{ver}"
patch_name += f"_epsilon={epsilon}_lambda1={lambda_1}_lambda2={lambda_2}"

uap_phantom_sponge_attack = UAPPhantomSponge(patch_folder=patch_name, 
                                             train_loader=train_loader, 
                                             val_loader=val_loader, 
                                             epsilon = epsilon, 
                                             lambda_1=lambda_1, 
                                             lambda_2=lambda_2, 
                                             #epochs=7,#stond hier voorheen nog niet
                                             patch_size=patch_size, 
                                             models_vers=models_vers)
adv_img = uap_phantom_sponge_attack.run_attack()

Fusing layers... 


CUDA Available:  False
local_yolos/yolov5/weights/yolov5s.pt


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
yolov5s summary: 270 layers, 7235389 parameters, 0 gradients, 16.6 GFLOPs


Epoch:  0
saving png of current patch at epoch: 0, batch: 0/169...
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =               0.435|0.249|0.019|0.0
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =               0.48|0.248|0.023|0.001
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =               0.432|0.248|0.018|0.002
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =               0.475|0.249|0.023|0.005
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =               0.46|0.248|0.021|0.005
saving png of current patch at epoch: 0, batch: 5/169...
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =               0.454|0.249|0.021|0.004
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =               0.431|0.249|0.018|0.007
BB CSs == 0.25 0
combined_loss, max_obj, bb_area, iou_loss for this batch =       

Traceback (most recent call last):
  File "/Users/coenschoof/miniconda/envs/phantomsponges/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/zm/yghhx9n94lj7sdcj5tgl1f9r0000gn/T/ipykernel_97157/2638083634.py", line 20, in <module>
    adv_img = uap_phantom_sponge_attack.run_attack()
  File "/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/attack/uap_phantom_sponge.py", line 678, in run_attack
    adv_patch = (torch.clip(patch + perturbation * factor, min_x, max_x))  # .detach()
  File "/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/attack/uap_phantom_sponge.py", line 626, in pgd_L2
    self.current_orig_classification_loss = 0.0
  File "/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/attack/uap_phantom_sponge.py", line 553, in fastGradientSignMethod
    self.models[r].zero_grad()
  File "/Users/coenschoof/miniconda/envs/phantomsponge